In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation
from keras.preprocessing.image import ImageDataGenerator
import h5py
import os
import cv2
import tensorflow as tf
import numpy as np

In [ ]:

#Training Data Directory
train_data_dir = ''

#Validation Data Directory
valid_data_dir = ''

#Test Data Directory
test_data_dir = ''

#Default image input size for VGG 16
img_width, img_height = 224,224

In [ ]:
#Initialize VGG 16 with ImageNet Weights
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet')

In [ ]:
vgg16_model.summary()

In [ ]:
#Create a new Sequential Model
model = Sequential()

#Add all VGG 16 except the last layer
for layer in vgg16_model.layers[:-1]:
    
    model.add(layer)
    
for layer in model.layers:
    
    #Freeze existing layers as they are pre-trained
    layer.trainable = False
    
#Add Output/Prediction layer for fine tuneing
model.add(Dense(1,activation='sigmoid'))

In [ ]:
#New model
model.summary()

In [ ]:
#Create an Image Data Generator  
datagen = ImageDataGenerator(rescale=1./255)

#Data Generator for training
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary',
        shuffle='False')

#Data Generator for Validation
validation_generator = datagen.flow_from_directory(
        valid_data_dir,
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode='binary',
        shuffle='False')

#Data Generator for Test data
test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode=None,
        shuffle='False')

In [ ]:
#Compile the model
model.compile(optimizer=keras.optimizers.SGD(lr=1e-4, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=672,
        nb_epoch=5,
        validation_data=validation_generator,
        validation_steps=79)

In [ ]:
#Save weights to directory
model.save_weights('')

In [ ]:
#Load weights from directory
model.load_weights('')

In [ ]:
#Evaluate model
#Also used to verify loaded weights
model.evaluate_generator(validation_generator,steps=79)

# Predict on test data (0 = No Smoke,1 = Smoke)

In [ ]:
model.predict_generator(test_generator,steps=79)